In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [112]:
import datetime


pontos_de_corte = 0.01

def merge_df(df_articles: pd.DataFrame, df_indices: pd.DataFrame) -> pd.DataFrame:
    
    clean_texts = [] 
    dt_temp = df_indices.index.min()

    df_articles = df_articles.sort_values('date').copy()

    for index, row in df_articles.iterrows():

        if not np.isnan(df_indices.reindex([row.date]).COEF_ANTERIOR[0]):
            dt_temp = row.date

        dt = dt_temp

        indice = resolve_indice(df_indices, dt)

        row_data = [dt, row.title, row.text, indice, df_indices.reindex([dt]).COMPARATIVO[0], row.link, df_indices.reindex([dt]).COEF[0], df_indices.reindex([dt]).COEF_ANTERIOR[0]]
        if('sobre_petrobras_diretamente' in df_articles.keys()):
            row_data.append(row.sobre_petrobras_diretamente)

        clean_texts.append(row_data)
        
    header = ['date', 'title', 'text', 'indice', 'comparativo', 'link', 'coef', 'coef_anterior']

    if('sobre_petrobras_diretamente' in df_articles.keys()):
        header.append('sobre_petrobras_diretamente')

    return pd.DataFrame(clean_texts, columns=header)

def resolve_indice(df_indices, dt):
    if df_indices.reindex([dt]).COMPARATIVO[0] >= pontos_de_corte:
        return 1
    elif df_indices.reindex([dt]).COMPARATIVO[0] <= (pontos_de_corte * -1):
        return -1
    else:
        return 0

In [94]:
df_articles = pd.read_csv('articles_per_date.csv', sep='\t', header=0)
df_articles_sem_merge = pd.read_csv('articles_por_data_sem_merge.csv', sep='\t', header=0)
df_articles_etiquetados = pd.read_csv('articles_por_data_etiquetados.csv', sep='\t', header=0)

In [95]:
df_articles_sem_merge.head()

,Unnamed: 0,title,text,date,category,subcategory,link,date_str
0,2,Três reportagens da Folha ganham Prêmio Petrob...,tres reportagens folha vencedoras premio petro...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017-09-10
1,18,Bolsa recua pelo 2º dia com feriado nos EUA; d...,bolsa brasileira fechou baixa nesta segunda mo...,2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...,2017-09-10
2,21,Moro pede que Lula esclareça se tem os recibos...,juiz sergio moro quer expresidente luiz inacio...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017-09-10
3,76,Lula na memória e no coração,ano eleicoes candidatura expresidente luiz ina...,2017-09-10,opiniao,NaN,http://www1.folha.uol.com.br/opiniao/2017/10/1...,2017-09-10
4,152,Destruindo a ciência brasileira,neste domingo ia escrever sobre premio nobel f...,2017-08-10,colunas,marcelogleiser,http://www1.folha.uol.com.br/colunas/marcelogl...,2017-08-10


Convert para data

In [96]:
df_datas = pd.to_datetime(df_articles.date)
df_articles.date = df_datas

df_datas = pd.to_datetime(df_articles_sem_merge.date)
df_articles_sem_merge.date = df_datas

In [37]:
df_articles_sem_merge.date.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11353 entries, 0 to 11352
Series name: date
Non-Null Count  Dtype         
--------------  -----         
11353 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 88.8 KB


Importa índices

In [38]:
df_indices = pd.read_excel('indices_per_date.xlsx', index_col=0)

In [97]:
np.isnan(df_indices.reindex(['2015-01-02']).COEF_ANTERIOR[0])

False

In [98]:
df_indices.index.min(), df_indices.index.max()

(Timestamp('2015-01-02 00:00:00'), Timestamp('2022-06-29 00:00:00'))

In [99]:
df_articles_sem_merge.head()

,Unnamed: 0,title,text,date,category,subcategory,link,date_str
0,2,Três reportagens da Folha ganham Prêmio Petrob...,tres reportagens folha vencedoras premio petro...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017-09-10
1,18,Bolsa recua pelo 2º dia com feriado nos EUA; d...,bolsa brasileira fechou baixa nesta segunda mo...,2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...,2017-09-10
2,21,Moro pede que Lula esclareça se tem os recibos...,juiz sergio moro quer expresidente luiz inacio...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017-09-10
3,76,Lula na memória e no coração,ano eleicoes candidatura expresidente luiz ina...,2017-09-10,opiniao,NaN,http://www1.folha.uol.com.br/opiniao/2017/10/1...,2017-09-10
4,152,Destruindo a ciência brasileira,neste domingo ia escrever sobre premio nobel f...,2017-08-10,colunas,marcelogleiser,http://www1.folha.uol.com.br/colunas/marcelogl...,2017-08-10


In [100]:
df_articles_sem_merge.to_csv('labeled_articles_sem_merge.csv', sep='\t')

In [101]:
df_articles_etiquetados.head()

,Unnamed: 0,date,title,text,link,sobre_petrobras_diretamente
0,1480,2016-05-12,"Após dois cortes, Petrobras eleva preços da ga...",mal consumidor sentiu bolso impacto duas reduc...,http://www1.folha.uol.com.br/mercado/2016/12/1...,Relacionado à Petrobrás
1,1481,2016-05-12,Liminar da Justiça de Sergipe suspende venda d...,vara justica federal sergipe determinou suspen...,http://www1.folha.uol.com.br/mercado/2016/12/1...,Relacionado ao mercado de ações
2,1483,2016-05-12,Dólar recua mais de 1% após protestos pouparem...,real maiores valorizacoes principais moedas ne...,http://www1.folha.uol.com.br/mercado/2016/12/1...,Relacionado ao mercado de ações
3,1491,2016-04-12,Metade das empresas não tem caixa para pagar d...,prolongada crise economica pais atravessa mina...,http://www1.folha.uol.com.br/mercado/2016/12/1...,Não relacionado ou não relacionado diretamente
4,1646,2016-05-11,Governo busca executivo experiente para o coma...,sob pressao pmdb camara deputados trocar coman...,http://www1.folha.uol.com.br/mercado/2016/11/1...,Não relacionado ou não relacionado diretamente


In [102]:
df_articles_sem_merge.date.min(), df_articles_sem_merge.date.max()

(Timestamp('2015-01-01 00:00:00'), Timestamp('2017-10-01 00:00:00'))

In [103]:
# df_merged = merge_df(df_articles, df_indices)

In [113]:
df_sem_merge_dos_artigos = merge_df(df_articles_sem_merge, df_indices)

In [114]:
df_sem_merge_dos_artigos[df_sem_merge_dos_artigos.coef.isna()]

,date,title,text,indice,comparativo,link,coef,coef_anterior


In [115]:
df_sem_merge_dos_artigos[df_sem_merge_dos_artigos.coef.isna()].shape, df_sem_merge_dos_artigos.shape

((0, 8), (11353, 8))

In [116]:
len(df_sem_merge_dos_artigos[df_sem_merge_dos_artigos.indice == 1]), len(df_sem_merge_dos_artigos[df_sem_merge_dos_artigos.indice == -1]), len(df_sem_merge_dos_artigos[df_sem_merge_dos_artigos.indice == 0])

(5437, 5590, 326)

In [119]:
df_sem_merge_dos_artigos[df_sem_merge_dos_artigos.title.str.contains('etrobras')].shape, df_sem_merge_dos_artigos.shape

((1927, 8), (11353, 8))

In [117]:
df_sem_merge_dos_artigos.to_csv('labeled_articles_sem_merge_data.csv', sep='\t', index=True)